# Chapter 5 -- SciPy and Statistical Computing
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/CH05_SciPy_Statistical_Computing.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Part:** 3 -- Machine Learning and AI  
**Prerequisites:** Chapter 4 (Data Visualisation)  
**Estimated time:** 3-4 hours

---

### Learning Objectives

By the end of this chapter you will be able to:

- Understand why statistical grounding matters before building ML models
- Use `scipy.stats` to describe distributions and test hypotheses
- Run t-tests, Mann-Whitney U tests, and chi-squared tests on real data
- Interpret p-values correctly -- and understand their limitations
- Use `scipy.optimize` for curve fitting and root finding
- Apply `scipy.interpolate` for filling gaps in time-series data
- Use NumPy structured arrays for mixed-type tabular data

---

### Why Statistics Before Machine Learning?

Many practitioners skip from data cleaning straight to model training.
This leads to models that are technically running but statistically meaningless --
trained on features that have no real relationship with the target,
or evaluated on metrics that mask fundamental data problems.

This chapter gives you the tools to ask: *Is this relationship real, or just noise?*
That question should precede every modelling decision.

---

### Project Thread -- Chapter 5

We run formal hypothesis tests on the SO 2025 dataset:
- Is the salary difference between Python and non-Python developers statistically significant?
- Does education level have a measurable effect on salary?
- Is there a significant salary gap between remote and on-site workers?

The answers inform which features are worth including in the Chapter 6 ML pipeline.


---

## Setup -- Imports and Data


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from scipy import stats
from scipy import optimize
from scipy import interpolate
import warnings
warnings.filterwarnings('ignore')

import scipy
print(f'SciPy:      {scipy.__version__}')
print(f'NumPy:      {np.__version__}')
print(f'Pandas:     {pd.__version__}')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi']     = 110
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.titleweight'] = 'bold'

DATASET_URL = 'https://raw.githubusercontent.com/timothy-watt/python-for-ai-ml/main/data/so_survey_2025_curated.csv'


In [ ]:
# Load and clean SO 2025 -- same pipeline as Chapters 3 and 4
df_raw = pd.read_csv(DATASET_URL)
df = df_raw.copy()
df = df.dropna(subset=['ConvertedCompYearly'])
df['ConvertedCompYearly'] = pd.to_numeric(df['ConvertedCompYearly'], errors='coerce')
Q1, Q3 = df['ConvertedCompYearly'].quantile([0.25, 0.75])
IQR = Q3 - Q1
df = df[
    (df['ConvertedCompYearly'] >= max(Q1 - 3*IQR, 5_000)) &
    (df['ConvertedCompYearly'] <= min(Q3 + 3*IQR, 600_000))
].copy()
if 'YearsCodePro' in df.columns:
    df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')
    df['YearsCodePro'] = df['YearsCodePro'].fillna(df['YearsCodePro'].median())
for col in ['Country', 'DevType', 'EdLevel', 'Employment', 'RemoteWork']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')
df['uses_python'] = df.get('LanguageHaveWorkedWith', pd.Series(dtype=str)).str.contains('Python', na=False)
df = df.reset_index(drop=True)
print(f'Dataset ready: {len(df):,} rows')


---

## Section 5.1 -- Describing Distributions with scipy.stats

Before testing hypotheses, you need to understand the shape of your data.
`scipy.stats` provides distribution fitting, descriptive statistics beyond
what Pandas offers, and the statistical tests that follow.


In [ ]:
# 5.1.1 -- Descriptive statistics and distribution shape

sal = df['ConvertedCompYearly'].dropna()

# scipy.stats.describe gives a comprehensive summary in one call
desc = stats.describe(sal)
print('scipy.stats.describe() output:')
print(f'  n:          {desc.nobs:,}')
print(f'  min/max:    ${desc.minmax[0]:,.0f} / ${desc.minmax[1]:,.0f}')
print(f'  mean:       ${desc.mean:,.0f}')
print(f'  variance:   {desc.variance:,.0f}')
print(f'  skewness:   {desc.skewness:.4f}')
print(f'  kurtosis:   {desc.kurtosis:.4f}')

# Interpreting skewness and kurtosis
print()
print('Interpretation:')
if desc.skewness > 0.5:
    print(f'  Skewness {desc.skewness:.2f} > 0.5 -> right-skewed (long tail of high earners)')
    print('  -> Consider log-transforming salary before regression modelling')
if desc.kurtosis > 1:
    print(f'  Kurtosis {desc.kurtosis:.2f} > 1 -> heavier tails than normal distribution')
    print('  -> More extreme outliers than a normal distribution would predict')

# Log-transform and compare skewness
log_sal = np.log(sal)
log_skew = stats.skew(log_sal)
print(f'  Original skewness:      {stats.skew(sal):.4f}')
print(f'  Log-transform skewness: {log_skew:.4f}  (closer to 0 = more symmetric)')


In [ ]:
# 5.1.2 -- Visualising the distribution and testing for normality

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

sal = df['ConvertedCompYearly'].dropna()
log_sal = np.log(sal)

# Left: raw salary histogram with normal fit overlay
axes[0].hist(sal, bins=60, density=True, color='#2E75B6', alpha=0.7, edgecolor='white')
x = np.linspace(sal.min(), sal.max(), 300)
mu, sigma = stats.norm.fit(sal)
axes[0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal fit')
axes[0].xaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f'${v/1000:.0f}k'))
axes[0].set_title('Raw Salary vs Normal Fit')
axes[0].set_xlabel('Salary')
axes[0].set_ylabel('Density')
axes[0].legend()

# Middle: log-salary histogram with normal fit overlay
axes[1].hist(log_sal, bins=60, density=True, color='#2E75B6', alpha=0.7, edgecolor='white')
lmu, lsigma = stats.norm.fit(log_sal)
lx = np.linspace(log_sal.min(), log_sal.max(), 300)
axes[1].plot(lx, stats.norm.pdf(lx, lmu, lsigma), 'r-', linewidth=2, label='Normal fit')
axes[1].set_title('Log(Salary) vs Normal Fit')
axes[1].set_xlabel('log(Salary)')
axes[1].set_ylabel('Density')
axes[1].legend()

# Right: Q-Q plot -- compares quantiles of data vs theoretical normal
# Points on the diagonal line = data follows a normal distribution
# Deviations at the tails = heavier or lighter tails than normal
sample_log = log_sal.sample(n=min(2000, len(log_sal)), random_state=42)
(osm, osr), (slope, intercept, r) = stats.probplot(sample_log, dist='norm')
axes[2].scatter(osm, osr, alpha=0.3, s=8, color='#2E75B6')
axes[2].plot([osm[0], osm[-1]],
             [slope*osm[0]+intercept, slope*osm[-1]+intercept],
             'r-', linewidth=2)
axes[2].set_title('Q-Q Plot: log(Salary) vs Normal')
axes[2].set_xlabel('Theoretical Quantiles')
axes[2].set_ylabel('Sample Quantiles')

plt.suptitle('SO 2025: Salary Distribution Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Shapiro-Wilk normality test on a sample (test requires n < 5000)
sample = sal.sample(n=500, random_state=42)
stat, p = stats.shapiro(sample)
print(f'Shapiro-Wilk test on raw salary (n=500):  W={stat:.4f}, p={p:.2e}')
stat_log, p_log = stats.shapiro(np.log(sample))
print(f'Shapiro-Wilk test on log salary  (n=500): W={stat_log:.4f}, p={p_log:.2e}')
print()
print('Interpretation:')
print('  p < 0.05 -> reject the null hypothesis that the data is normally distributed')
print('  Even log-salary is not perfectly normal at this sample size -- this is common.')
print('  For large n, use non-parametric tests (Mann-Whitney U) as a safer alternative.')


---

## Section 5.2 -- Hypothesis Testing

A hypothesis test asks: *Could this observed difference have occurred by chance?*

The framework:
1. **H0 (null hypothesis):** No real difference -- any observed gap is due to random sampling
2. **H1 (alternative hypothesis):** A real difference exists
3. **p-value:** Probability of observing data this extreme *if H0 were true*
4. **Significance level (alpha):** Threshold below which we reject H0. Convention: 0.05

**Critical p-value warning:** p < 0.05 does NOT mean the effect is large or practically
important. With 15,000 respondents, tiny meaningless differences can be statistically
significant. Always report **effect size** alongside the p-value.


In [ ]:
# 5.2.1 -- Two-sample t-test: Python vs non-Python salary
#
# Question: Is the mean salary of Python developers different from non-Python developers?
# H0: mu_python == mu_non_python  (no difference)
# H1: mu_python != mu_non_python  (two-tailed: we do not assume direction)

if 'uses_python' in df.columns:
    python_sal  = df[df['uses_python']]['ConvertedCompYearly'].dropna()
    nopython_sal = df[~df['uses_python']]['ConvertedCompYearly'].dropna()

    print(f'Python developers:     n={len(python_sal):,}, '
          f'mean=${python_sal.mean():,.0f}, median=${python_sal.median():,.0f}')
    print(f'Non-Python developers: n={len(nopython_sal):,}, '
          f'mean=${nopython_sal.mean():,.0f}, median=${nopython_sal.median():,.0f}')
    print(f'Observed difference:   ${python_sal.mean() - nopython_sal.mean():+,.0f} (mean)')

    # Independent samples t-test
    # equal_var=False -> Welch's t-test, does not assume equal variances
    # This is the safer default for real-world data
    t_stat, p_val = stats.ttest_ind(python_sal, nopython_sal, equal_var=False)

    print()
    print('Welch t-test result:')
    print(f'  t-statistic: {t_stat:.4f}')
    print(f'  p-value:     {p_val:.4e}')
    print(f'  Significant at alpha=0.05: {p_val < 0.05}')

    # Cohen's d -- effect size (how large is the difference relative to spread?)
    # < 0.2: negligible, 0.2-0.5: small, 0.5-0.8: medium, > 0.8: large
    pooled_std = np.sqrt(
        (python_sal.std()**2 + nopython_sal.std()**2) / 2
    )
    cohens_d = (python_sal.mean() - nopython_sal.mean()) / pooled_std
    print(f"  Cohen's d:   {cohens_d:.4f}  ", end='')
    if abs(cohens_d) < 0.2:   print('(negligible effect size)')
    elif abs(cohens_d) < 0.5: print('(small effect size)')
    elif abs(cohens_d) < 0.8: print('(medium effect size)')
    else:                      print('(large effect size)')

    print()
    print('Conclusion:')
    if p_val < 0.05:
        print(f'  We reject H0. The salary difference is statistically significant.')
        print(f'  However, Cohen\'s d={cohens_d:.2f} -- interpret practical significance carefully.')
    else:
        print('  We fail to reject H0. No statistically significant salary difference.')


In [ ]:
# 5.2.2 -- Mann-Whitney U test: a non-parametric alternative
#
# The t-test assumes approximate normality. Since salary is skewed,
# the Mann-Whitney U test is a safer choice -- it makes no distribution assumptions.
# It tests whether one group tends to have higher values than the other.

if 'uses_python' in df.columns:
    u_stat, p_mw = stats.mannwhitneyu(
        python_sal, nopython_sal,
        alternative='two-sided'   # two-sided: we test for any difference
    )

    # Effect size for Mann-Whitney: rank-biserial correlation
    # Ranges from -1 to +1; 0 = no difference
    n1, n2 = len(python_sal), len(nopython_sal)
    r_rb = 1 - (2 * u_stat) / (n1 * n2)

    print('Mann-Whitney U test (non-parametric):')
    print(f'  U-statistic:          {u_stat:,.0f}')
    print(f'  p-value:              {p_mw:.4e}')
    print(f'  Significant (a=0.05): {p_mw < 0.05}')
    print(f'  Rank-biserial r:      {r_rb:.4f}')

    print()
    print('Both t-test and Mann-Whitney U tell the same story -- good.')
    print('When they disagree, prefer Mann-Whitney U for skewed data.')


In [ ]:
# 5.2.3 -- One-way ANOVA: salary differences across education levels
#
# ANOVA (Analysis of Variance) tests whether means differ across 3+ groups.
# H0: all group means are equal
# H1: at least one group mean is different

if 'EdLevel' in df.columns:
    ed_map = {
        "Bachelor's degree (B.A., B.S., B.Eng., etc.)": "Bachelor's",
        "Master's degree (M.A., M.S., M.Eng., MBA, etc.)": "Master's",
        'Some college/university study without earning a degree': 'Some college',
        'Other doctoral degree (Ph.D., Ed.D., etc.)': 'PhD',
        'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 'Secondary',
    }
    df['ed_short'] = df['EdLevel'].replace(ed_map)

    # Keep the 5 most common education levels
    top_ed = df['ed_short'].value_counts().head(5).index.tolist()
    groups = [
        df[df['ed_short'] == ed]['ConvertedCompYearly'].dropna().values
        for ed in top_ed
    ]

    f_stat, p_anova = stats.f_oneway(*groups)

    print('One-way ANOVA: salary by education level')
    print(f'  Groups tested: {top_ed}')
    print(f'  F-statistic:   {f_stat:.4f}')
    print(f'  p-value:       {p_anova:.4e}')
    print(f'  Significant:   {p_anova < 0.05}')

    print()
    print('Group medians:')
    for ed, grp in zip(top_ed, groups):
        print(f'  {ed:<20} n={len(grp):>5,}  median=${np.median(grp):>10,.0f}')

    # Eta-squared: effect size for ANOVA
    # Proportion of total variance explained by group membership
    all_vals = np.concatenate(groups)
    grand_mean = all_vals.mean()
    ss_between = sum(len(g) * (g.mean() - grand_mean)**2 for g in groups)
    ss_total   = sum((v - grand_mean)**2 for v in all_vals)
    eta_sq = ss_between / ss_total
    print(f'  Eta-squared: {eta_sq:.4f}  ({eta_sq*100:.1f}% of salary variance explained by education)')


In [ ]:
# 5.2.4 -- Chi-squared test: independence of categorical variables
#
# Chi-squared tests whether two categorical variables are INDEPENDENT.
# H0: the variables are independent (no association)
# H1: the variables are associated
#
# Here: is Python adoption independent of remote work status?

if 'RemoteWork' in df.columns and 'uses_python' in df.columns:
    top_remote = df['RemoteWork'].value_counts().head(3).index.tolist()
    df_sub = df[df['RemoteWork'].isin(top_remote)].copy()

    # Build a contingency table: rows=remote status, cols=uses Python
    contingency = pd.crosstab(df_sub['RemoteWork'], df_sub['uses_python'])
    contingency.columns = ['Non-Python', 'Python']
    print('Contingency table (RemoteWork x Python usage):')
    print(contingency)

    chi2, p_chi, dof, expected = stats.chi2_contingency(contingency)

    print()
    print('Chi-squared test:')
    print(f'  chi2 statistic: {chi2:.4f}')
    print(f'  p-value:        {p_chi:.4e}')
    print(f'  Degrees of freedom: {dof}')
    print(f'  Significant:    {p_chi < 0.05}')

    # Cramer's V: effect size for chi-squared
    n = contingency.values.sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency.shape) - 1)))
    print(f"  Cramer's V:     {cramers_v:.4f}  ", end='')
    if cramers_v < 0.1:   print('(negligible association)')
    elif cramers_v < 0.3: print('(weak association)')
    elif cramers_v < 0.5: print('(moderate association)')
    else:                  print('(strong association)')


In [ ]:
# 5.2.5 -- Visualising the hypothesis test results

if 'uses_python' in df.columns and 'RemoteWork' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Left: salary distributions for Python vs non-Python
    for uses_py, label, colour in [
        (True,  f'Python (n={len(python_sal):,})',     '#E8722A'),
        (False, f'Non-Python (n={len(nopython_sal):,})', '#2E75B6')
    ]:
        data = df[df['uses_python'] == uses_py]['ConvertedCompYearly'].dropna()
        axes[0].hist(data, bins=50, density=True, alpha=0.5, color=colour, label=label)
        axes[0].axvline(data.median(), color=colour, linestyle='--', linewidth=2)
    axes[0].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x/1000:.0f}k'))
    axes[0].set_title(f'Salary: Python vs Non-Python\n(p={p_mw:.2e}, Mann-Whitney U)')
    axes[0].set_xlabel('Annual Salary (USD)')
    axes[0].legend(fontsize=9)

    # Right: median salary by education level
    if 'ed_short' in df.columns:
        ed_medians = (
            df[df['ed_short'].isin(top_ed)]
            .groupby('ed_short')['ConvertedCompYearly']
            .median().sort_values(ascending=True)
        )
        axes[1].barh(ed_medians.index, ed_medians.values, color='#2E75B6', edgecolor='white')
        axes[1].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x/1000:.0f}k'))
        for i, v in enumerate(ed_medians.values):
            axes[1].text(v + 500, i, f'${v/1000:.0f}k', va='center', fontsize=9)
        axes[1].set_title(f'Median Salary by Education\n(ANOVA p={p_anova:.2e}, eta^2={eta_sq:.3f})')
        axes[1].set_xlabel('Median Annual Salary (USD)')

    plt.suptitle('SO 2025: Hypothesis Test Results', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


---

## Section 5.3 -- scipy.optimize: Curve Fitting and Root Finding

`scipy.optimize` solves mathematical optimisation problems. Two uses matter most
for data science:

- **Curve fitting:** find the parameters of a function that best fit your data
- **Root finding:** find where a function equals zero (useful for threshold analysis)

Understanding curve fitting conceptually prepares you for neural network training
in Chapter 7 -- gradient descent is just a more sophisticated optimiser doing
the same thing: minimising a loss function by adjusting parameters.


In [ ]:
# 5.3.1 -- Curve fitting: model salary growth with experience
#
# We fit a logarithmic curve to the experience-salary relationship.
# Logarithmic growth makes intuitive sense: early career years have
# a large salary impact; later years have diminishing returns.

if 'YearsCodePro' in df.columns:
    # Bin experience into groups and compute median salary per group
    exp_bins = list(range(0, 26, 2))   # 0,2,4,...,24
    df['exp_bin'] = pd.cut(df['YearsCodePro'], bins=exp_bins, labels=exp_bins[:-1])
    exp_salary = (
        df.groupby('exp_bin', observed=True)['ConvertedCompYearly']
        .agg(['median', 'count'])
        .reset_index()
    )
    exp_salary.columns = ['years', 'median_salary', 'count']
    exp_salary = exp_salary[exp_salary['count'] >= 20].copy()
    exp_salary['years'] = exp_salary['years'].astype(float) + 1   # avoid log(0)

    x_data = exp_salary['years'].values
    y_data = exp_salary['median_salary'].values

    # Define the model function to fit
    # a * log(x) + b: logarithmic growth with two free parameters
    def log_model(x, a, b):
        return a * np.log(x) + b

    # curve_fit finds a and b that minimise sum of squared residuals
    popt, pcov = optimize.curve_fit(log_model, x_data, y_data)
    a_fit, b_fit = popt

    # R-squared: goodness of fit
    y_pred   = log_model(x_data, *popt)
    ss_res   = np.sum((y_data - y_pred) ** 2)
    ss_tot   = np.sum((y_data - y_data.mean()) ** 2)
    r_squared = 1 - ss_res / ss_tot

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.scatter(x_data, y_data, s=exp_salary['count']/5,
               color='#2E75B6', alpha=0.8, label='Observed median salary')
    x_smooth = np.linspace(x_data.min(), x_data.max(), 200)
    ax.plot(x_smooth, log_model(x_smooth, *popt), 'r-', linewidth=2.5,
            label=f'Fit: {a_fit:,.0f} * log(x) + {b_fit:,.0f}  (R^2={r_squared:.3f})')
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f'${v/1000:.0f}k'))
    ax.set_xlabel('Years of Professional Experience')
    ax.set_ylabel('Median Annual Salary (USD)')
    ax.set_title('SO 2025: Salary vs Experience -- Logarithmic Curve Fit')
    ax.legend(fontsize=10)
    plt.tight_layout()
    plt.show()

    print(f'Fitted model: salary = {a_fit:,.0f} * log(years) + {b_fit:,.0f}')
    print(f'R-squared: {r_squared:.4f}  ({r_squared*100:.1f}% of variance explained)')
    for yrs in [1, 3, 5, 10, 15, 20]:
        pred = log_model(yrs, *popt)
        print(f'  Predicted median at {yrs:>2} years: ${pred:>10,.0f}')


In [ ]:
# 5.3.2 -- Root finding: find experience level at a salary threshold
#
# Root finding answers: at what experience level does the model predict
# a salary of exactly $X? We solve log_model(x) - target = 0.

if 'YearsCodePro' in df.columns:
    targets = [80_000, 100_000, 120_000, 150_000]

    print('Experience years needed to reach salary targets (model prediction):')
    for target in targets:
        # Define f(x) = model(x) - target; find x where f(x) = 0
        def equation(x):
            return log_model(x, a_fit, b_fit) - target

        try:
            # brentq: reliable bracketed root-finding method
            # We search between 0.5 and 40 years
            root = optimize.brentq(equation, 0.5, 40)
            print(f'  ${target:>10,}  ->  {root:.1f} years experience')
        except ValueError:
            print(f'  ${target:>10,}  ->  outside model range')


---

## Section 5.4 -- scipy.interpolate: Filling Gaps

`scipy.interpolate` estimates values between known data points.
The most common use in data science is filling gaps in time series data
or smoothing noisy measurements. We demonstrate with a synthetic salary
trend over time -- the same technique applies to sensor data, financial
time series, or any sequential dataset with missing observations.


In [ ]:
# 5.4.1 -- Spline interpolation on salary trend data

# Simulate annual median salary data with some missing years
years_known  = np.array([2017, 2018, 2020, 2021, 2023, 2024, 2025])
salary_known = np.array([85_000, 90_000, 98_000, 105_000, 118_000, 125_000, 132_000])

# Cubic spline: smooth curve that passes exactly through all known points
spline = interpolate.CubicSpline(years_known, salary_known)

# Linear interpolation: straight lines between known points
linear = interpolate.interp1d(years_known, salary_known, kind='linear')

years_fine = np.linspace(2017, 2025, 200)   # 200 evenly spaced points

fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(years_known, salary_known, s=80, color='#1B3A5C', zorder=5,
           label='Known data points')
ax.plot(years_fine, spline(years_fine),  color='#E8722A', linewidth=2.5,
        label='Cubic spline interpolation')
ax.plot(years_fine, linear(years_fine),  color='#2E75B6', linewidth=1.5,
        linestyle='--', label='Linear interpolation')

# Mark the gap years that were filled
gap_years = np.array([2019, 2022])
for yr in gap_years:
    spline_val = float(spline(yr))
    ax.scatter([yr], [spline_val], s=100, color='#E8722A', marker='D', zorder=6)
    ax.annotate(f'Filled:\n${spline_val/1000:.0f}k',
                xy=(yr, spline_val), xytext=(yr+0.15, spline_val-4000),
                fontsize=8.5, color='#E8722A')

ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f'${v/1000:.0f}k'))
ax.set_xlabel('Year')
ax.set_ylabel('Median Annual Salary (USD)')
ax.set_title('Salary Trend Interpolation: Cubic Spline vs Linear')
ax.legend(fontsize=10)
plt.tight_layout()
plt.show()

print('Filled missing years with cubic spline:')
for yr in gap_years:
    print(f'  {yr}: ${float(spline(yr)):,.0f}')
print()
print('Cubic spline is smoother (captures curvature); linear is simpler and never overshoots.')
print('Use linear for data with sharp changes; cubic for smooth continuous trends.')


---

## Section 5.5 -- Project: Statistical Feature Selection Summary

We now have statistical evidence for which features are related to salary.
This directly informs the feature engineering decisions in Chapter 6.


In [ ]:
print('=' * 65)
print('  SO 2025: Statistical Analysis Summary')
print('  Feature Relevance for Salary Prediction')
print('=' * 65)

results = []

if 'uses_python' in df.columns:
    results.append({
        'Feature':    'Python usage',
        'Test':       'Mann-Whitney U',
        'p-value':    f'{p_mw:.2e}',
        'Effect':     f'r={r_rb:.3f} (rank-biserial)',
        'Include':    'YES' if p_mw < 0.05 else 'NO'
    })

if 'EdLevel' in df.columns:
    results.append({
        'Feature':    'Education level',
        'Test':       'One-way ANOVA',
        'p-value':    f'{p_anova:.2e}',
        'Effect':     f'eta^2={eta_sq:.3f} ({eta_sq*100:.1f}% variance)',
        'Include':    'YES' if p_anova < 0.05 else 'NO'
    })

if 'RemoteWork' in df.columns:
    results.append({
        'Feature':    'Remote work',
        'Test':       'Chi-squared',
        'p-value':    f'{p_chi:.2e}',
        'Effect':     f"Cramer's V={cramers_v:.3f}",
        'Include':    'YES' if p_chi < 0.05 else 'NO'
    })

if 'YearsCodePro' in df.columns:
    results.append({
        'Feature':    'Years experience',
        'Test':       'Curve fit (R^2)',
        'p-value':    'N/A',
        'Effect':     f'R^2={r_squared:.3f} ({r_squared*100:.1f}% variance)',
        'Include':    'YES' if r_squared > 0.5 else 'MARGINAL'
    })

print(f'{"Feature":<20} {"Test":<20} {"p-value":<12} {"Effect size":<30} {"Include"}')
print('-' * 90)
for r in results:
    print(f'{r["Feature"]:<20} {r["Test"]:<20} {r["p-value"]:<12} {r["Effect"]:<30} {r["Include"]}')

print()
print('All tested features show statistically significant relationships with salary.')
print('All will be included in the Chapter 6 scikit-learn pipeline.')
print('Remember: statistical significance does not guarantee predictive power --')
print('that is what cross-validated model evaluation in Chapter 6 measures.')


---

## Concept Check Questions

> Test your understanding before moving on. Answer each question without referring back to the notebook, then expand to check.

**Q1.** What is a **p-value**, and what does p = 0.03 tell you — and not tell you?

<details><summary>Show answer</summary>

A p-value is the probability of seeing results at least this extreme *if the null hypothesis were true*. p = 0.03 gives grounds to reject the null. It does **not** tell you the probability the null is true, the effect size, or whether the result is practically meaningful.

</details>

**Q2.** What is Cohen's d, and why should you always report it alongside a p-value?

<details><summary>Show answer</summary>

Cohen's d measures effect size — the mean difference divided by the pooled standard deviation. With large samples, trivially small differences become statistically significant. Cohen's d (small ≈ 0.2, medium ≈ 0.5, large ≈ 0.8) tells you whether the finding is practically meaningful.

</details>

**Q3.** When should you use a Mann-Whitney U test instead of a t-test?

<details><summary>Show answer</summary>

When the data is non-normal (heavy tails, outliers) or ordinal. The Mann-Whitney U test is non-parametric — it compares rank orderings, not means, making it robust when normality cannot be assumed.

</details>

**Q4.** What does `scipy.optimize.curve_fit` return?

<details><summary>Show answer</summary>

`(popt, pcov)`. `popt` is the array of optimal parameter values. `pcov` is the covariance matrix — its diagonal's square roots give the standard errors, quantifying uncertainty in each parameter estimate.

</details>

**Q5.** Python developers earn $12k more (p < 0.001, Cohen's d = 0.18). How would you explain this to a business stakeholder?

<details><summary>Show answer</summary>

The difference is statistically real but the effect size is small (d = 0.18 is below the conventional 'small' threshold of 0.2). The $12k gap exists, but experience, country, and role likely matter far more. Don't over-interpret it as a strong causal signal.

</details>



---

## Coding Exercises

> Three exercises per chapter: **🔧 Guided** (fill-in-the-blanks) · **🔨 Applied** (write from scratch) · **🏗️ Extension** (go beyond the chapter)

Exercises use the SO 2025 developer survey dataset.
Expand each **Solution** block only after attempting the exercise.


### Exercise 1 🔧 Guided — Salary distribution hypothesis test

Complete the `compare_salary_groups()` function that:
1. Tests whether `uses_python=1` earns significantly more than `uses_python=0`
2. Checks normality of each group with Shapiro-Wilk
3. Chooses Mann-Whitney U (non-normal) or independent t-test (normal)
4. Returns a dict with `statistic`, `p_value`, `test_used`, `significant` (α=0.05)


In [ ]:
from scipy import stats
import numpy as np

rng = np.random.default_rng(42)
group0 = np.exp(10.6 + rng.normal(0,0.5,500))
group1 = np.exp(10.9 + rng.normal(0,0.5,500))

def compare_salary_groups(g0: np.ndarray, g1: np.ndarray, alpha: float = 0.05) -> dict:
    # 1. Shapiro-Wilk normality test (use sample of 500)
    _, p_norm0 = stats.shapiro(g0[:500])
    _, p_norm1 = None, None  # YOUR CODE
    both_normal = None  # YOUR CODE
    # 2. Choose and run appropriate test
    if both_normal:
        stat, p = None, None  # YOUR CODE (t-test)
        test_name = 'ttest'
    else:
        stat, p = None, None  # YOUR CODE (Mann-Whitney U)
        test_name = 'mannwhitneyu'
    return {'statistic': round(stat,4), 'p_value': round(p,6),
            'test_used': test_name, 'significant': p < alpha}

print(compare_salary_groups(group0, group1))

<details><summary>💡 Hint</summary>

`stats.ttest_ind(g0, g1, alternative='less')` for one-tailed.
`stats.mannwhitneyu(g0, g1, alternative='less')` for non-parametric.
`both_normal = p_norm0 > 0.05 and p_norm1 > 0.05`

</details>

<details><summary>✅ Solution</summary>

```python
def compare_salary_groups(g0, g1, alpha=0.05):
    _, p0 = stats.shapiro(g0[:500]); _, p1 = stats.shapiro(g1[:500])
    both_normal = p0 > 0.05 and p1 > 0.05
    if both_normal:
        stat, p = stats.ttest_ind(g0, g1); test_name='ttest'
    else:
        stat, p = stats.mannwhitneyu(g0, g1, alternative='two-sided'); test_name='mannwhitneyu'
    return {'statistic': round(stat,4), 'p_value': round(p,6), 'test_used': test_name, 'significant': p<alpha}
```

</details>


### Exercise 2 🔨 Applied — Effect size across all binary features

For each binary feature in the SO 2025 dataset (`uses_python`, `uses_sql`,
`uses_js`, `uses_ai`), compute:
- Mann-Whitney U p-value (salary: feature=1 vs feature=0)
- Cohen's d effect size
- Rank the features by effect size and plot as a horizontal bar chart


In [ ]:
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(42)
n = 5000
df = pd.DataFrame({
    'salary':     np.exp(10.8 + rng.normal(0,0.5,n)),
    'uses_python':rng.integers(0,2,n),
    'uses_sql':   rng.integers(0,2,n),
    'uses_js':    rng.integers(0,2,n),
    'uses_ai':    rng.integers(0,2,n)})

def cohens_d(g1: np.ndarray, g2: np.ndarray) -> float:
    # YOUR CODE
    pass

# YOUR CODE: compute results for all 4 features and plot

<details><summary>💡 Hint</summary>

Cohen's d = (mean1 - mean2) / pooled_std
pooled_std = sqrt(((n1-1)*std1² + (n2-1)*std2²) / (n1+n2-2))
Filter df by feature value to get each group's salary distribution.

</details>

<details><summary>✅ Solution</summary>

```python
def cohens_d(g1, g2):
    n1,n2 = len(g1),len(g2)
    pooled = np.sqrt(((n1-1)*g1.std()**2+(n2-1)*g2.std()**2)/(n1+n2-2))
    return (g1.mean()-g2.mean())/pooled
results=[]
for feat in ['uses_python','uses_sql','uses_js','uses_ai']:
    g1=df[df[feat]==1]['salary'].values; g0=df[df[feat]==0]['salary'].values
    _,p=stats.mannwhitneyu(g1,g0,alternative='two-sided')
    d=cohens_d(g1,g0)
    results.append({'feature':feat,'p_value':p,'cohens_d':d})
res=pd.DataFrame(results).sort_values('cohens_d')
res.plot.barh(x='feature',y='cohens_d'); plt.title('Cohen\'s d Effect Size on Salary'); plt.show()
```

</details>


### Exercise 3 🏗️ Extension — Bootstrap confidence intervals

Implement `bootstrap_ci(data, statistic_fn, n_bootstrap=10000, ci=0.95)`
that returns a (lower, upper) confidence interval for any statistic.

Use it to compute 95% CIs for the median salary of each education level.
Plot the results as a forest plot (point estimate + error bars).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bootstrap_ci(
    data: np.ndarray,
    statistic_fn,
    n_bootstrap: int = 10000,
    ci: float = 0.95,
    seed: int = 42,
) -> tuple[float, float]:
    # YOUR CODE
    pass

# Test
salaries = np.exp(10.8 + np.random.normal(0, 0.5, 500))
low, high = bootstrap_ci(salaries, np.median)
print(f'Median: {np.median(salaries):,.0f}  95% CI: [{low:,.0f}, {high:,.0f}]')

<details><summary>💡 Hint</summary>

Draw `n_bootstrap` samples with replacement using `rng.choice(data, size=len(data), replace=True)`.
Apply `statistic_fn` to each bootstrap sample. The CI is the `(1-ci)/2` and `(1+ci)/2` percentiles of the bootstrap distribution.

</details>

<details><summary>✅ Solution</summary>

```python
def bootstrap_ci(data, statistic_fn, n_bootstrap=10000, ci=0.95, seed=42):
    rng = np.random.default_rng(seed)
    bootstraps = []
    for _ in range(n_bootstrap):
        sample = rng.choice(data,size=len(data),replace=True)
        bootstraps.append(statistic_fn(sample))
    bootstraps = np.array(bootstraps)
    alpha = (1-ci)/2
    return (np.percentile(bootstraps, 100*alpha), np.percentile(bootstraps, 100*(1-alpha)))
```

</details>


---

## Chapter 5 Summary

### Key Takeaways

- **Describe before modelling:** skewness, kurtosis, and Q-Q plots reveal whether
  your data meets modelling assumptions. Salary is right-skewed -- log-transform it.
- **t-test** is the standard two-group mean comparison. Use **Welch's version**
  (`equal_var=False`) by default -- it does not assume equal variances.
- **Mann-Whitney U** is the non-parametric alternative. Safer for skewed data.
  Use it when you cannot justify normality assumptions.
- **ANOVA** extends the t-test to 3+ groups. Follow with a post-hoc test
  (Tukey HSD) to find which specific pairs differ.
- **Chi-squared** tests independence between two categorical variables.
- **p-value alone is not enough.** Always report an effect size:
  Cohen's d (t-test), rank-biserial r (Mann-Whitney), eta-squared (ANOVA),
  Cramer's V (chi-squared).
- **curve_fit** finds optimal parameters for any user-defined function.
  The same idea -- minimising residuals -- underlies all of ML.
- **Root finding** with `brentq` inverts a fitted model to answer threshold questions.
- **Spline interpolation** fills gaps smoothly; linear interpolation is safer
  when you cannot assume smooth curvature.

### Project Thread Status

| Test | Feature | Result |
|------|---------|--------|
| Mann-Whitney U | Python usage vs salary | Statistically significant |
| One-way ANOVA | Education level vs salary | Statistically significant |
| Chi-squared | Remote work vs Python usage | Tested |
| Curve fit | Experience vs salary | Logarithmic model fitted |
| Feature selection | All four features | Included in Chapter 6 pipeline |

---

### What's Next: Chapter 6 -- Machine Learning with scikit-learn

Chapter 6 is the largest chapter in the book. We build a complete scikit-learn
pipeline: preprocessing, feature engineering, salary regression, developer role
classification, and developer clustering. The statistical groundwork from this
chapter informs every feature decision we make there.

---

*End of Chapter 5 -- Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
